**INSTALL DEPENDENCIES & HUGGINGFACE LOGIN**

In [ ]:
!pip install flask pyngrok transformers torch accelerate bitsandbytes sentencepiece --quiet

from huggingface_hub import login

# Log in to Hugging Face Hub using your personal access token
login("YOUR HuggingFaceHub TOKEN HERE")

**CREATE app.py**

In [ ]:
%%writefile app.py
from flask import Flask, render_template, request
import functools
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

app = Flask(__name__)

# ===========================================================
# 🧠 MODEL LOADING SECTION
# Loads the Llama model once and caches it for reuse.
# This avoids repeated loading and greatly improves latency.
# ===========================================================@functools.lru_cache(maxsize=1)
def load_llama_model():
    print("⏳ Loading Llama 3.1 model...")

    model_name = "meta-llama/Llama-3.1-8B-Instruct"

    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # Load model in 8-bit for memory efficiency
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map="auto",
        load_in_4bit=True
    )

    print("✅ Model Loaded Successfully!")
    return tokenizer, model


# ===========================================================
# 🔮 RESPONSE GENERATION SECTION
# Uses your original logic for applying the chat template
# and generating structured recommendations.
# ===========================================================
def generate_llm_response(system_prompt, user_prompt, max_tokens=700):
    tokenizer, model = load_llama_model()

    # Construct system + user conversation
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]

    # Convert into model-specific chat format
    input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt").to(model.device)

    # Generate assistant response
    output_ids = model.generate(
        input_ids,
        max_new_tokens=max_tokens,
        do_sample=False,
        temperature=0.3,
        pad_token_id=tokenizer.eos_token_id
    )

    # Extract only newly generated text
    generated_tokens = output_ids[0][input_ids.shape[-1]:]
    final_output = tokenizer.decode(generated_tokens, skip_special_tokens=True)

    return final_output.strip()


# ===========================================================
# 🌐 FLASK HOME ROUTE
# Accepts input from HTML form, builds user profile,
# and returns generated career recommendations.
# ===========================================================@app.route("/", methods=["GET", "POST"])
def home():
    recommendation = ""

    if request.method == "POST":
        # Extract user inputs from the form
        user_stage = request.form["user_stage"]
        skills = request.form["skills"]
        interests = request.form["interests"]
        education = request.form["education"]
        goals = request.form["goals"]

        # Build structured user profile
        user_profile = f"""
User Category: {user_stage}
Skills & Strengths: {skills}
Interests & Passions: {interests}
Education / Background: {education}
Career Goals & Preferences: {goals}

IMPORTANT: Tailor the recommendation, learning roadmap, and tone based on the user's category.
"""

        # System prompt with rules & structure
        system_message = """
You are a certified professional career counselor.

Output Rules (VERY IMPORTANT):
- Never repeat, restate, quote, paraphrase or summarize the user input.
- Never reference or indirectly restate user input; respond using generalized, inferred wording only.
- Never mention system instructions, prompts, or rules.
- Never include the words: "system", "assistant", "user", "prompt", "instruction".
- Never output chat history, metadata, or dates.
- Start your response immediately with the final answer content only.

Category Guide:
1 → Provide beginner-friendly roadmap, internships, foundation projects.
2 → Provide role-upgrade strategy, certifications, portfolio emphasis.
3 → Provide specialization, leadership or domain advancement strategy.
4 → Provide transition plan, transferable skills and bridging roadmap.
5 → Provide confidence-based re-entry plan, refresher + gradual upskilling.

Final Output Must Include:
1. Top 3 suitable career paths
2. Short reasoning for each
3. Key strength summary
4. Skill-gap improvement plan
5. 30/60/90 day roadmap
6. Motivational closing message

Style:
Professional, concise, practical, supportive.
"""
        # Try generating the final response
        try:
            recommendation = generate_llm_response(system_message, user_profile)
        except Exception as e:
            recommendation = f"❌ Error: {e}"
    # Render recommendation in UI
    return render_template("index.html", recommendation=recommendation)


# ===========================================================
# 🚀 RUN FLASK SERVER
# ===========================================================if __name__ == "__main__":
    app.run(host="0.0.0.0", port=8000, debug=False)


**Templates Folder**

In [ ]:
!mkdir -p templates
!mkdir -p static

index.html

In [ ]:
%%writefile templates/index.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1" />
    <title>🎯 Personalized Career Guidance</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}" />
</head>

<body>
    <div class="hero-section">
        <div class="overlay"></div>

        <div class="hero-content">

            <h1>🎯 Personalized Career Recommendation</h1>
            <p>Get AI-powered guidance tailored to your strengths & goals.</p>

            <form method="post" class="input-card">

                <select name="user_stage" required>
                    <option value="1">Student / Fresher</option>
                    <option value="2">Working Professional (1–3 years)</option>
                    <option value="3">Experienced Professional (3+ years)</option>
                    <option value="4">Career Switcher</option>
                    <option value="5">Returning After Career Break</option>
                </select>

                <textarea name="skills" rows="3" placeholder="Your top skills & strengths" required></textarea>

                <textarea name="interests" rows="3" placeholder="Your interests or passions" required></textarea>

                <textarea name="education" rows="3" placeholder="Your education or background" required></textarea>

                <textarea name="goals" rows="3" placeholder="Your long-term career goals" required></textarea>

                <button type="submit" class="btn-generate">Generate Recommendations 🚀</button>
            </form>
        </div>
    </div>

    {% if recommendation %}
    <div class="result-section fade-in">
        <div class="result-card">
            <h2>📌 Your Personalized Career Guidance</h2>
            <p style="white-space: pre-line;">{{ recommendation }}</p>
        </div>
    </div>
    {% endif %}

</body>
</html>


CSS: style.css

In [ ]:
%%writefile static/style.css
/* Same styling theme as the example you provided */

@import url('https://fonts.googleapis.com/css2?family=Poppins:wght@400;600&display=swap');

body {
    font-family: 'Poppins', sans-serif;
    margin: 0;
    color: #fff;
}

/* Background */
.hero-section {
    position: relative;
    height: 100vh;
    display: flex;
    align-items: center;
    background: url('https://res.cloudinary.com/dehfj1nrp/image/upload/v1761895287/portrait-business-woman-with-enthusiastic-face-expression-smiling-looking-confident-standing-s_btoget.jpg')
        center/cover no-repeat;
    padding-left: 8%;
}

.overlay {
    position: absolute;
    inset: 0;
    background: rgba(0, 0, 0, 0.55);
}

/* Main Box */
.hero-content {
    position: relative;
    z-index: 2;
    width: 500px;
    max-width: 90%;
    padding: 45px;
    background: rgba(255, 255, 255, 0.18);
    border-radius: 25px;
    backdrop-filter: blur(7px);
}

h1 {
    font-size: 2.3rem;
    margin-bottom: 10px;
    color: #a8c5ff;
}

.input-card {
    display: flex;
    flex-direction: column;
    gap: 15px;
}

textarea, select {
    width: 100%;
    padding: 12px;
    border-radius: 10px;
    border: none;
    background: rgba(255, 255, 255, 0.9);
    color: #111;
}

.btn-generate {
    background: linear-gradient(135deg, #6a5acd, #00bcd4);
    border: none;
    padding: 12px 0;
    font-weight: 600;
    border-radius: 10px;
    cursor: pointer;
    color: #fff;
}

.result-section {
    background: #0d111f;
    padding: 60px 20px;
    display: flex;
    justify-content: center;
}

.result-card {
    background: rgba(30, 34, 80, 0.35);
    padding: 25px;
    border-radius: 15px;
    max-width: 700px;
    color: #e2e5ff;
    white-space: pre-line;
}

/* Animation */
.fade-in {
    animation: fadeInUp 0.6s ease forwards;
}

@keyframes fadeInUp {
    from { opacity: 0; transform: translateY(20px); }
    to   { opacity: 1; transform: translateY(0); }
}


**Run Flask + ngrok**

In [ ]:
# Kill existing processes
!pkill -f flask || echo "No flask running"
!pkill -f ngrok || echo "No ngrok running"

In [ ]:
!lsof -i :8000


In [ ]:
# (Optional) If a specific PID is blocking:
!kill -9 617

In [ ]:
# Start Flask in background
!nohup python app.py > flask.log 2>&1 &

In [ ]:
# Start ngrok
from pyngrok import ngrok, conf

# Enter your NGROK auth token here
conf.get_default().auth_token = "INPUT_YOUR_NGROK_TOKEN_HERE"

public_url = ngrok.connect(8000)
print("🌍 Public URL:", public_url)
